<a href="https://colab.research.google.com/github/jfahadi/Android-Project/blob/master/federate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow tensorflow-federated
!pip install tensorflow tensorflow-federated wfdb numpy pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of tensorflow-federated to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 105.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You 

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of wfdb to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 8.4 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import tensorflow_federated as tff
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import wfdb
import os
from sklearn.model_selection import train_test_split
from google.colab import files
from sklearn.preprocessing import LabelEncoder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#upoladed = files.upload()  # Upload CSV file
#data = pd.read_csv("your_dataset.csv")
#print(data.head())  # Inspect the first few rows
data_path = "/content/drive/MyDrive/mit-bih-arrhythmia-database/"
record_file = os.path.join(data_path, "100")  # Replace with the specific record file
annotation_file = os.path.join(data_path, "100")

#print(os.listdir('/content/drive/MyDrive/mit-bih-arrhythmia-database/'))

# Load the record and annotation
record = wfdb.rdrecord(record_file)
annotation = wfdb.rdann(annotation_file, 'atr')

# Extract the ECG signal and labels
ecg_signal = record.p_signal[:, 0]  # First channel
labels = annotation.symbol  # Annotation symbols
#Normalize the ECG Signal

scaler = StandardScaler()
ecg_signal = scaler.fit_transform(ecg_signal.reshape(-1, 1)).flatten()

#Segment the signal
def segment_signal(signal, labels, window_size=300):
    segments = []
    segment_labels = []
    for i in range(0, len(signal) - window_size, window_size):
        segments.append(signal[i:i + window_size])
        segment_labels.append(labels[i // window_size])  # Simplified label per segment
    return np.array(segments), np.array(segment_labels)

X, y = segment_signal(ecg_signal, labels)

#Encode Labels

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)



# Define the model
def create_rnn_model(input_shape, output_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=input_shape),
        tf.keras.layers.SimpleRNN(128, activation='relu', return_sequences=False),
        tf.keras.layers.Dense(output_size, activation='softmax')
    ])
    return model

# Model function for TFF
def model_fn():
    input_spec = {
        'x': tf.TensorSpec(shape=(None, 28, 28), dtype=tf.float32),
        'y': tf.TensorSpec(shape=(None,), dtype=tf.int32),
    }
    keras_model = create_rnn_model(input_shape=(28, 28), output_size=10)
    return tff.learning.models.from_keras_model(
        keras_model=keras_model,
        input_spec=input_spec,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    )

# Define optimizer functions
client_optimizer_fn = lambda: tf.keras.optimizers.Adam(learning_rate=0.001)
server_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate=1.0)


# Build the Federated Averaging algorithm
iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn=model_fn,
    client_optimizer_fn=client_optimizer_fn,
    server_optimizer_fn=server_optimizer_fn
)

# Initialize the federated process
state = iterative_process.initialize()

# Simulated federated data
federated_data = [
    tf.data.Dataset.from_tensor_slices({
        'x': tf.random.uniform((10, 28, 28)),  # Example input data
        'y': tf.random.uniform((10,), maxval=10, dtype=tf.int32)  # Example labels
    }).batch(2)
]

# Federated training loop
NUM_ROUNDS = 10
for round_num in range(1, NUM_ROUNDS + 1):
    state, metrics = iterative_process.next(state, federated_data)
    print(f"Round {round_num}, Metrics: {metrics}")


ERROR:jax._src.xla_bridge:Jax plugin configuration error: Plugin module %s could not be loaded
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/xla_bridge.py", line 428, in discover_pjrt_plugins
    plugin_module = importlib.import_module(plugin_module_name)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1204, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1176, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1147, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 690, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_r

Mounted at /content/drive


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/mit-bih-arrhythmia-database/100.hea'

# Function to Set Weights in Keras Model

In [ ]:
def set_model_weights(model, iterative_process, tff_state):
    """
    Load TFF model weights into a Keras model.
    Parameters:
        - model: A Keras model instance.
        - iterative_process: The TFF iterative process.
        - tff_state: LearningAlgorithmState object from TFF.
    """
    # Access the model weights using get_model_weights() on iterative_process
    tff_weights = iterative_process.get_model_weights(tff_state)

    # Assign the TFF weights to the Keras model
    tff_weights.assign_weights_to(model)

# Generate Synthetic Test Data

In [ ]:
def generate_synthetic_time_series_data(num_samples, timesteps, num_features, output_size):
    """
    Generate synthetic time-series data for evaluation.
    Parameters:
        - num_samples: Number of samples.
        - timesteps: Number of timesteps per sample.
        - num_features: Number of features per timestep.
        - output_size: Number of output classes (for classification).
    """
    X = np.random.rand(num_samples, timesteps, num_features)
    y = np.random.randint(0, output_size, size=(num_samples,))
    return X, y

# Define input shape and output size
input_shape = (28, 28)  # Input shape matching federated training
output_size = 10        # Number of output classes

# Load the Global Weights into a New Keras Model

In [ ]:
rnn_model = create_rnn_model(input_shape=input_shape, output_size=output_size)
set_model_weights(rnn_model, iterative_process, state)

# Generate Synthetic Test Data
X_test, y_test = generate_synthetic_time_series_data(100, input_shape[0], input_shape[1], output_size)





In [ ]:
rnn_model = create_rnn_model(input_shape=input_shape, output_size=output_size)
set_model_weights(rnn_model, iterative_process, state)

# Compile the model before evaluation
rnn_model.compile(optimizer='adam', # You can choose any suitable optimizer
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# Generate Synthetic Test Data
X_test, y_test = generate_synthetic_time_series_data(100, input_shape[0], input_shape[1], output_size)

# Evaluate Global Model
eval_metrics = rnn_model.evaluate(X_test, y_test, verbose=2)
print("Evaluation Metrics:", eval_metrics)

4/4 - 1s - loss: 2.8768 - accuracy: 0.1300 - 548ms/epoch - 137ms/step
Evaluation Metrics: [2.876816749572754, 0.12999999523162842]


save model

In [ ]:
# Save the model to a file
#rnn_model.save("global_rnn_model.h5")
#print("Model saved to 'global_rnn_model.h5'")
